In [1]:
import os
import csv
train_file=os.path.join('AAAI_dataset', 'politi_train.csv')
test_file=os.path.join('AAAI_dataset', 'politi_test.csv')
def read_csv(file):
    dataset=[]
    with open('{}'.format(file), encoding="utf-8") as f:
        reader = csv.reader(f)
        for idx,row in enumerate(reader):
            if idx >= 1:
                text = row[1]
                img_name=row[2]
                label=row[3]
                if label == '0':
                    dataset.append((text, img_name, 0))
                elif label=='1':
                    dataset.append((text, img_name, 1))
    return dataset
train_dataset=read_csv(train_file)
test_dataset=read_csv(test_file)

In [2]:
len(train_dataset),len(test_dataset)


(381, 104)

In [3]:
label_set=[[],[]]
for idx, data in enumerate(train_dataset):
    _,_,label=data
    if label==0:
        label_set[0].append(idx)
    else:
        label_set[1].append(idx)
print(label_set)
print(len(label_set[0]))
print(len(label_set[1]))

[[0, 1, 10, 14, 24, 27, 30, 33, 36, 38, 39, 48, 49, 62, 63, 65, 70, 72, 75, 76, 77, 78, 79, 81, 83, 84, 86, 91, 93, 96, 101, 108, 109, 110, 114, 116, 117, 118, 122, 123, 128, 131, 132, 135, 137, 138, 139, 141, 142, 143, 147, 151, 154, 156, 157, 165, 166, 168, 169, 176, 178, 179, 185, 186, 187, 188, 189, 190, 193, 194, 196, 197, 200, 201, 203, 204, 206, 211, 212, 215, 220, 223, 225, 226, 228, 231, 238, 241, 242, 250, 251, 253, 256, 258, 261, 268, 269, 271, 277, 278, 279, 286, 287, 292, 293, 298, 301, 303, 304, 305, 312, 318, 319, 323, 324, 325, 337, 338, 341, 343, 344, 345, 346, 351, 353, 356, 361, 363, 364, 367, 369, 370, 372, 373, 377], [2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 25, 26, 28, 29, 31, 32, 34, 35, 37, 40, 41, 42, 43, 44, 45, 46, 47, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 64, 66, 67, 68, 69, 71, 73, 74, 80, 82, 85, 87, 88, 89, 90, 92, 94, 95, 97, 98, 99, 100, 102, 103, 104, 105, 106, 107, 111, 112, 113, 115, 119, 120, 121, 124, 125, 1

In [4]:
train_dataset[0]

('Democrats go full TYRANNY: Now demand nationwide gun confiscation from law-abiding Americans… at gunpoint, of course by: Ethan Huff\n\nNaturalNews.com\n\nMonday, May 07, 2018\n\nA California Democrat was recently given a platform by USA Today to publish a shocking editorial that calls for nationwide confiscation of all “military-style semiautomatic assault weapons” from law-abiding citizens.\n\nRepresentative Eric Swalwell from California’s 15th District wrote that all so-called “assault” weapons need to be banned, and that a federal gun buy-back program needs to be instituted in order to effectively collect them all from the citizenry.\n\nRep. Swalwell even goes a step further, insisting that those who refuse to hand over their “assault” weapons be criminally prosecuted – including law-abiding gun owners who have never been convicted of committing a crime with their legally-purchased weaponry.\n\nNot content to simply impose a fresh ban on all new “assault” weapon purchases, which i

In [5]:
test_dataset[0]

('Philando Castile Had Been Stopped 52 Times By Police – WCCO \n\n\n\n— When Philando Castile saw the flashing lights in his rearview mirror the night he got shot, it wasn’t unusual. He had been pulled over at least 52 times in recent years in and around the Twin Cities and given citations for minor offenses including speeding, driving without a muffler and not wearing a seat belt.\n\nHe was assessed at least $6,588 in fines and fees, although more than half of the total 86 violations were dismissed, court records show.\n\nWas Castile an especially bad driver or just unlucky? Or was he targeted by officers who single out black motorists like him for such stops, as several of his family members have alleged?\n\nThe answer may never be known, but Castile’s stop for a broken tail light Wednesday ended with him fatally shot by a suburban St. Paul police officer, and Castile’s girlfriend livestreaming the chilling aftermath.\n\nThe shooting has added a new impetus to a national debate on ra

In [6]:
import nltk
import torch
import torch.nn as nn
from transformers import XLNetTokenizer, XLNetModel
from conf import model_config_bert as model_config
class LinearEmbed(nn.Module):
    """Embedding module"""
    def __init__(self, dim_in=128, dim_out=2):
        super(LinearEmbed, self).__init__()
        self.linear = nn.Linear(dim_in, dim_out)
        # self.l2norm = Normalize(2)

    def forward(self, x):
        x = x.view(x.shape[0], -1)
        x = self.linear(x)
        # x = self.l2norm(x)
        return x
tokenizer = XLNetTokenizer.from_pretrained("data/pretrain_model/twitter/xlnet-base-cased")
model_text = XLNetModel.from_pretrained("data/pretrain_model/twitter/xlnet-base-cased")
text_classifer=LinearEmbed(15360,2)
model_text.to('cuda')
text_classifer.to('cuda')
def text_embedding(text, text_model_path=None):
    emb_set=[]
    sentences = nltk.sent_tokenize(text)
    if text_model_path:
        
        checkpoint_text = torch.load(text_model_path)
        model_text.load_state_dict(checkpoint_text['model'])
    for sent in sentences:
        input_ids = torch.tensor(tokenizer.encode(sent)).unsqueeze(0)
        input_ids = input_ids.to('cuda')
        outputs= model_text(input_ids)
        cls_output = torch.mean(outputs[0], axis=1)
        emb_set.append(cls_output)
        if len(emb_set)>=20:
            emb_set=emb_set[:20]
        else:
            deficit=20-len(emb_set)
            for i in range(deficit):
                emb_set.append(torch.zeros((1,768)).cuda())
    emb_set = torch.cat(emb_set, dim=0)
    emb_set= torch.flatten(emb_set,0).unsqueeze(0)
    text_logist=text_classifer(emb_set)
    return emb_set,text_logist
        
    
    

In [7]:
tex,_,_=train_dataset[0]
emb,logist=text_embedding(tex)
print(emb.size(),logist.size())




torch.Size([1, 15360]) torch.Size([1, 2])


In [8]:
from PIL import Image
from PIL import ImageFile
import torchvision.transforms as transforms
from model.resnet import resnet50
transform_train = transforms.Compose([     
#             transforms.Resize((256, 128), interpolation=3),
        transforms.Resize((224,224), interpolation=3),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
transform_test = transforms.Compose([     
        transforms.Resize((224,224), interpolation=3),  # Image.BICUBIC
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
model_img = resnet50(pretrained=True,num_classes=2)
model_img.to('cuda')
def img_embedding(img_name,is_train, img_model_path=None):
    if is_train:
        img_path= 'AAAI_dataset/Images/politi_train/'+img_name
        img = Image.open(img_path).convert('RGB')   
        res_img=transform_train(img).float().unsqueeze(0).float()
    else:
        img_path= 'AAAI_dataset/Images/politi_test/'+img_name
        img = Image.open(img_path).convert('RGB')   
        res_img=transform_test(img).float().unsqueeze(0).float()   
    if img_model_path:
        checkpoint = torch.load(img_model_path)
        model_img.load_state_dict(checkpoint['model'])
    res_img=res_img.to('cuda')
    img_feat,img_logist=model_img(res_img)
    return img_feat,img_logist


        

In [9]:
# _,train_img,_=train_dataset[0]
# _,test_img,_=test_dataset[0]
# train_img_feat, train_img_logist=img_embedding(train_img,is_train=True)
# test_img_feat, test_img_logist=img_embedding(test_img,is_train=False)
# train_img_feat.size(), train_img_logist.size(), test_img_feat.size(), test_img_logist.size()

In [10]:

from crd.criterion import CRDLoss
import torch.optim as optim
criterion_cls = nn.CrossEntropyLoss()

criterion_cls=criterion_cls.cuda()

optimizer_img = optim.SGD(model_img.parameters(), lr=0.0003, momentum=0.9, weight_decay=5e-4)
optimizer_text = torch.optim.Adam(model_text.parameters(), lr=2e-5)   #1e-5, 2e-5, 3e-5

text_module_list = nn.ModuleList([])
text_module_list.append(model_text)
text_module_list.append(text_classifer)


ModuleList(
  (0): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (1): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)


In [11]:
import torch
import random
import copy
import numpy as np


# def img_embedding_batch(batch_data, is_train):
#     batch_img_logist, batch_label=[],[]
#     for idx, data in enumerate(batch_data):
#         _, img_name, label=data
#         _,img_item_logist = img_embedding(img_name,is_train=is_train)
#         batch_img_logist.append(img_item_logist)
#         batch_label.append(label)

#     batch_img_logist=torch.cat(batch_img_logist,0)
#     batch_label=torch.tensor(batch_label)
    
#     return batch_img_logist, batch_label



    
        
        
        
        
    
        


In [12]:
from helper import *

# def train_img_epoch(epoch, train_set):
#     model_img.train()
#     model_img.cuda()
#     losses_img = AverageMeter()
#     top1_img = AverageMeter()
#     cur_step=0
#     for i in range(0,384,16):  
#         cur_step+=1
#         if i<368:
#             batch_data=train_set[i:i+16]
#         else:
#             batch_data=train_set[i:]
#         batch_img_logist, batch_label = img_embedding_batch(batch_data, is_train=True)
#         batch_label=batch_label.cuda()
        
#         loss_cls_img = criterion_cls(batch_img_logist, batch_label)

#         acc1_img = accuracy(batch_img_logist, batch_label)
#         losses_img.update(loss_cls_img.item(), batch_label.size(0))
#         top1_img.update(float(acc1_img[0]), batch_label.size(0))

#         optimizer_img.zero_grad()
#         loss_cls_img.backward()
#         optimizer_img.step()

#         if cur_step % 3 == 0:
#             print('Train for Image Model:')
#             print('Epoch: [{0}]\t'
#                   'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
#                   'Acc@1 {top1.val:.3f} ({top1.avg:.3f})\t'.format(
#                 epoch,
#                 loss=losses_img, top1=top1_img))
#             print('\n')
#     print(' Train image model : Acc@1 {top1.avg:.3f}'.format(top1=top1_img))
    
# def test_img_epoch(epoch, train_set):
#     model_img.eval()
#     model_img.cuda()
#     losses_img = AverageMeter()
#     top1_img = AverageMeter()
#     cur_step=0
#     with torch.no_grad():
#         for i in range(0,112,16):  
#             cur_step += 1
#             if i<96:
#                 batch_data=test_set[i:i+16]
#             else:
#                 batch_data=test_set[i:]
    
#             batch_img_logist, batch_label = img_embedding_batch(batch_data, is_train=False)
#             batch_label=batch_label.cuda()

#             loss_cls_img = criterion_cls(batch_img_logist, batch_label)

#             acc1_img = accuracy(batch_img_logist, batch_label)
#             losses_img.update(loss_cls_img.item(), batch_label.size(0))
#             top1_img.update(float(acc1_img[0]), batch_label.size(0))

            

#             if cur_step % 3 == 0:
#                 print('Test for Image Model:')
#                 print('Epoch: [{0}]\t'
#                       'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
#                       'Acc@1 {top1.val:.3f} ({top1.avg:.3f})\t'.format(
#                     epoch,
#                     loss=losses_img, top1=top1_img))
#                 print('\n')
#         print(' Test image model : Acc@1 {top1.avg:.3f}'.format(top1=top1_img))
#         print('\n')
#         return top1_img.avg
        


In [13]:
# from conf import config
# best_img_acc = 0
# img_save_file = None
# train_set=copy.deepcopy(train_dataset)
# test_set=copy.deepcopy(test_dataset)
# sys.stdout = Logger(os.path.join(config.save_folder, 'img.txt'))
# for epoch_id in range(60):
#     print('Epoch:', epoch_id)
#     random.shuffle(train_set)
#     train_img_epoch(epoch_id, train_set)
#     test_img_acc = test_img_epoch(epoch_id,test_set)
#     if test_img_acc > best_img_acc:
#         best_img_acc = test_img_acc
#         state = {
#             'epoch': epoch_id,
#             'model': model_img.state_dict(),
#             'best_acc': best_img_acc,
#         }
#         img_save_file = os.path.join(config.save_folder, 'img_best.pth')
#         print('saving the best model for img!')
#         torch.save(state, img_save_file)
#         print('\n')
# print('best accuracy for image model :', best_img_acc)

In [14]:

def text_embedding_batch(batch_data):
    batch_text_logist,batch_label=[],[]
    for idx, data in enumerate(batch_data):
        text, _, label=data
        _,text_item_logist = text_embedding(text,text_model_path='/home/wzm/test/rumor_detect/data/log/xlnet-base-cased/politi/text_best.pth')
        batch_text_logist.append(text_item_logist)
        batch_label.append(label)

    batch_text_logist=torch.cat(batch_text_logist,0)
    batch_label=torch.tensor(batch_label)
    
    return batch_text_logist, batch_label

In [15]:
from helper import *

def train_text_epoch(epoch, train_set):
    for module in text_module_list:
        module.train()
        module.cuda()
    losses_text = AverageMeter()
    top1_text = AverageMeter()
    cur_step=0
    for i in range(0,384,16):  
        cur_step+=1
        if i<368:
            batch_data=train_set[i:i+16]
        else:
            batch_data=train_set[i:]
        batch_text_logist, batch_label = text_embedding_batch(batch_data)
        batch_label=batch_label.cuda()
        
        loss_cls_text = criterion_cls(batch_text_logist, batch_label)

        acc1_text = accuracy(batch_text_logist, batch_label)
        losses_text.update(loss_cls_text.item(), batch_label.size(0))
        top1_text.update(float(acc1_text[0]), batch_label.size(0))

        optimizer_text.zero_grad()
        loss_cls_text.backward()
        optimizer_text.step()

        if cur_step % 3 == 0:
            print('Train for Text Model:')
            print('Epoch: [{0}]\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Acc@1 {top1.val:.3f} ({top1.avg:.3f})\t'.format(
                epoch,
                loss=losses_text, top1=top1_text))
            print('\n')
    print(' Train text model : Acc@1 {top1.avg:.3f}'.format(top1=top1_text))
    
def test_text_epoch(epoch, test_set):
    for module in text_module_list:
        module.eval()
        module.cuda()
    top1_text = AverageMeter()
    count_0_lists = []
    correct_0_lists = []
    target_0_lists = []
    count_1_lists = []
    correct_1_lists = []
    target_1_lists = []
    cur_step=0
    with torch.no_grad():
        for i in range(0,112,16):  
            cur_step += 1
            if i<96:
                batch_data=test_set[i:i+16]
            else:
                batch_data=test_set[i:]
    
            batch_text_logist, batch_label = text_embedding_batch(batch_data)
            batch_label=batch_label.cuda()

           

            acc1_text = accuracy(batch_text_logist, batch_label)
            
            count_0, count_correct_0, count_target_0 = metric(logist_fc, batch_label, for_fake=True)
            count_1, count_correct_1, count_target_1 = metric(logist_fc, batch_label, for_fake=False)
            count_0_lists.append(count_0)
            correct_0_lists.append(count_correct_0)
            target_0_lists.append(count_target_0)

            count_1_lists.append(count_1)
            correct_1_lists.append(count_correct_1)
            target_1_lists.append(count_target_1)

           
            top1_text.update(float(acc1_text[0]), batch_label.size(0))

            

            if cur_step % 3 == 0:
                print('Test for Text Model:')
                print('Epoch: [{0}]\t'
                      'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                      'Acc@1 {top1.val:.3f} ({top1.avg:.3f})\t'.format(
                    epoch,
                    loss=losses_text, top1=top1_text))
                print('\n')
        epoch_count_0 = sum(count_0_lists)
        epoch_correct_0 = sum(correct_0_lists)
        epoch_target_0 = sum(target_0_lists)

        epoch_count_1 = sum(count_1_lists)
        epoch_correct_1 = sum(correct_1_lists)
        epoch_target_1 = sum(target_1_lists)

        try:
            prec_0 = float(epoch_correct_0) * (100.0 / float(epoch_count_0))
        except ZeroDivisionError:
            prec_0 = 0
        try:
            rec_0 = float(epoch_correct_0) * (100.0 / float(epoch_target_0))
        except ZeroDivisionError:
            rec_0 = 0
        try:
            f_0 = 2 * (prec_0 * rec_0) / (prec_0 + rec_0)
        except ZeroDivisionError:
            f_0 = 0

        try:
            prec_1 = float(epoch_correct_1) * (100.0 / float(epoch_count_1))
        except ZeroDivisionError:
            prec_1 = 0
        try:
            rec_1 = float(epoch_correct_1) * (100.0 / float(epoch_target_1))
        except ZeroDivisionError:
            rec_1 = 0
        try:
            f_1 = 2 * (prec_1 * rec_1) / (prec_1 + rec_1)
        except ZeroDivisionError:
            f_1 = 0

        # prec_1 = float(epoch_correct_1) * (100.0 / float(epoch_count_1))
        # rec_1 = float(epoch_correct_1) * (100.0 / float(epoch_target_1))
        # f_1 = 2 * (prec_1 * rec_1) / (prec_1 + rec_1)
        print('val * Acc@1 {top1.avg:.3f} '.format(top1=top1_text))
        print('metrics: prec_0, rec_0, f_0, prec_1, rec_1, f_1')
        print(prec_0, rec_0, f_0, prec_1, rec_1, f_1)

    return top1_text.avg, prec_0, rec_0, f_0, prec_1, rec_1, f_1
        
#         print(' Test text model : Acc@1 {top1.avg:.3f}'.format(top1=top1_text))
#         print('\n')
#         return top1_text.avg
        

In [16]:
test_set=copy.deepcopy(test_dataset)
acc, prec_0, rec_0, f_0, prec_1, rec_1, f_1 = test_text_epoch(1,test_set)
print('full metric under best accuracy: acc, prec, rec, F1, prec_fake, rec_fake, F1_fake')
print(acc, prec_0, rec_0, f_0, prec_1, rec_1, f_1)

RuntimeError: Error(s) in loading state_dict for XLNetModel:
	Missing key(s) in state_dict: "mask_emb", "word_embedding.weight", "layer.0.rel_attn.q", "layer.0.rel_attn.k", "layer.0.rel_attn.v", "layer.0.rel_attn.o", "layer.0.rel_attn.r", "layer.0.rel_attn.r_r_bias", "layer.0.rel_attn.r_s_bias", "layer.0.rel_attn.r_w_bias", "layer.0.rel_attn.seg_embed", "layer.0.rel_attn.layer_norm.weight", "layer.0.rel_attn.layer_norm.bias", "layer.0.ff.layer_norm.weight", "layer.0.ff.layer_norm.bias", "layer.0.ff.layer_1.weight", "layer.0.ff.layer_1.bias", "layer.0.ff.layer_2.weight", "layer.0.ff.layer_2.bias", "layer.1.rel_attn.q", "layer.1.rel_attn.k", "layer.1.rel_attn.v", "layer.1.rel_attn.o", "layer.1.rel_attn.r", "layer.1.rel_attn.r_r_bias", "layer.1.rel_attn.r_s_bias", "layer.1.rel_attn.r_w_bias", "layer.1.rel_attn.seg_embed", "layer.1.rel_attn.layer_norm.weight", "layer.1.rel_attn.layer_norm.bias", "layer.1.ff.layer_norm.weight", "layer.1.ff.layer_norm.bias", "layer.1.ff.layer_1.weight", "lay

In [ ]:
# from conf import config
# best_text_acc = 0
# text_save_file = None
# train_set=copy.deepcopy(train_dataset)
# test_set=copy.deepcopy(test_dataset)
# sys.stdout = Logger(os.path.join(config.save_folder, 'text.txt'))
# for epoch_id in range(60):
#     print('Epoch:', epoch_id)
#     random.shuffle(train_set)
#     train_text_epoch(epoch_id, train_set)
#     test_text_acc = test_text_epoch(epoch_id,test_set)
#     if test_text_acc > best_text_acc:
#         best_text_acc = test_text_acc
#         state = {
#             'epoch': epoch_id,
#             'model': model_text.state_dict(),
#             'best_acc': best_text_acc,
#         }
#         text_save_file = os.path.join(config.save_folder, 'text_best.pth')
#         print('saving the best model for text!')
#         torch.save(state, text_save_file)
#         print('\n')
# print('best accuracy for text model :', best_text_acc)